# DI Pre Processing
Get desired filenames and store them in log files
<br>
<br>
Notes:
1) changes directories
2) update root path
<br>
<br>
![UofC logo](./pictures/uofc_logo-black.jpg)

In [1]:
#import libraries
import os
import re

In [2]:
# make sure to update path
root_path = f"C:/Christian/DI_centre_structured"
input(f"Is this the right directory - {root_path}?")

''

In [3]:
# get file directory
repo_dir = os.getcwd()

In [4]:
p3225_file_path = {
    "video folder" : "DI_CAMERA_P3225",
    "1st level" : ["Final"],
    "2nd level" : "Patient Name",
    "3rd level" : [2, 3],
    "4th level" : { "B": "With Blankets",
                        "WOB": "Without Blankets"}, 
    "5th level" : {"H" : "Hold Breath",
                    "R": "Relaxed"} 
}

fur_file_path = {
    "video folder" : "DI_THERMAL_FLIR",
    "1st level" : [""],
    "2nd level" : "Patient Number",
    "3rd level" : [2, 3],
    "4th level" : { "H": "Hold Breath",
                        "R": "Relaxed"}, 
    "5th level" : { "B": "With Blankets",
                        "WOB": "Without Blankets"} 
}

In [5]:
def get_video_filenames(patient_path:str, start_level:int, max_level:int) -> None:
    file_paths = []

    os.chdir(patient_path)
    if start_level < max_level:
        for entry in os.scandir(patient_path):
            if entry.is_file():
                file_paths.append(entry.path)
            elif entry.is_dir():
                file_paths.extend(get_video_filenames(entry.path, start_level+1, max_level))

    return file_paths

In [6]:
def get_patient_id(curr_dir:str) -> None:
    os.chdir(curr_dir)
    patient_names = [folder_content for folder_content in os.listdir() if os.path.isdir(folder_content)]
    return patient_names

In [7]:
def filter_filenames(video_filenames:list[str], keyword:str) -> list[str]:
    filtered_filenames = []
    for filename in video_filenames:
        if not re.search(r"\b{}\b".format(keyword), filename, flags=re.IGNORECASE):
            filtered_filenames.append(filename)
    return filtered_filenames

In [8]:
def save_filenames(root_dir:str, save_dir:str, log_filename:str, keywords:list[str], patient_list:list[str], max_level:int) -> None:
    log_file = f"{save_dir}/{log_filename}.log"
    log_file = log_file.replace("\\", "/")

    if os.path.exists(log_file):
        os.remove(log_file)

    for patient_count, patient_name in enumerate(patient_list):
        patient_folder = root_dir + "/" + patient_name
        all_patient_mp4_files = get_video_filenames(patient_folder, 0, max_level)

        for word in keywords:
            all_patient_mp4_files = filter_filenames(all_patient_mp4_files, word)

        with open(log_file, "a") as mp4_data:
            for mp4_file in all_patient_mp4_files:
                mp4_data.write(f"{mp4_file}\n")

## Retrieve all videos (stage 1.1)

Filter out unwanted filenames and save them in log files

In [9]:
""" 
    get filenames for rgb camera
"""

video_folder = p3225_file_path["video folder"]
all_videos_folder = root_path + "/" + video_folder + "/" + p3225_file_path["1st level"][0]
patient_names = get_patient_id(all_videos_folder)
patient_names

['Arun']

In [10]:
"save mp4 file paths in log file (rgb)"

records_folder = repo_dir + "/records/logs"
log_filename = video_folder + "-mp4"
keywords = ["media_media.amp-.mp4"]
search_level = 4
save_filenames(all_videos_folder, records_folder, log_filename, keywords, patient_names, search_level)

In [11]:
"save png file paths in log file (rgb)"

log_filename = video_folder + "-png"
keywords = ["media_media.amp-.mp4", ".mp4"]
search_level = 10
save_filenames(all_videos_folder, records_folder, log_filename, keywords, patient_names, search_level)

In [12]:
""" 
    get filenames for thermal camera
"""

video_folder = fur_file_path["video folder"]
all_videos_folder = root_path + "/" + video_folder + "/" + fur_file_path["1st level"][0]
patient_names = get_patient_id(all_videos_folder)
patient_names

FileNotFoundError: [WinError 2] The system cannot find the file specified: 'C:/Christian/DI_centre_structured/DI_THERMAL_FLIR/'

In [ ]:
"save avi file paths in log file (thermal)"

records_folder = repo_dir + "/records/logs"
# keywords = [".Bmp", ".png"]
log_filename = video_folder + "-avi"
keywords = [".Bmp"]
search_level = 4
save_filenames(all_videos_folder, records_folder, log_filename, keywords, patient_names, search_level)

In [ ]:
"save avi file paths in log file (thermal)"

log_filename = video_folder + "-png"
keywords = [".Bmp", ".avi"]
search_level = 10
save_filenames(all_videos_folder, records_folder, log_filename, keywords, patient_names, search_level)